In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:

# node for building the tree 
class TreeNode(): 
  # constructor
  def __init__(self, featureIndex=None, conditionVal=None, leftNode=None, rightNode=None, value=None):      
    self.featureIndex = featureIndex
    self.conditionVal = conditionVal
    self.leftNode = leftNode
    self.rightNode = rightNode
    self.value = value # only leaf node holds value

In [30]:
class MyDecisionTreeClassifier():

    # constructor
    def __init__(self, nMinNode, X, Y):
        self.nMinNode = nMinNode
        self.root = self.buildDecisionTreeRecursive(X,Y)

    # average value calculator
    def getAvgValues(self, xCol):
        n = len(xCol)
        meanVals = np.zeros(n)
        for i in range(n-1):
            meanVals[i] = (xCol[i]+xCol[i+1])/2
        return meanVals
    
    # calculate entropy
    def entropy(self, y):
        categories = np.unique(y)
        entropy = 0
        for catg in categories:
            prob = len(y[y == catg]) / len(y)
            entropy += -prob * np.log2(prob)
        return entropy

    # compute information gain
    def infoGain(self, p, l, r):
        return (self.entropy(p) - ((len(l) / len(p))*self.entropy(l) + (len(r) / len(p))*self.entropy(r)))

    # split the node
    def splitNode(self, X,y, m):
        
        # holds the information about current split
        retCondVal = None
        retFeatureIndx = None
        maxGain = -np.inf
        
        # for each feature
        for i in range(m):

            uniqueVals = np.unique(X[:, i])
            # possible condition values for making decisions
            possibleCondVal = self.getAvgValues(uniqueVals)

            # loop over all the feature values present in the data
            for cond in possibleCondVal:
                li = X[:, i] <= cond
                ri = X[:,i] > cond

                # check if it is required to split 
                if (len(li)!=0 and len(ri)!=0):
                    infoGain = self.infoGain(y, y[li], y[ri])
                    # update if gain is higher
                    if infoGain>maxGain:
                        retFeatureIndx = i
                        retCondVal = cond
                        maxGain = infoGain        
        return retCondVal, retFeatureIndx
    

    # build the decision tree    
    def buildDecisionTreeRecursive(self, X, y, depth=0):
        n, m = np.shape(X)
        # check if all y's are the same
        if(np.all(y==y[0])):
            return TreeNode(value=y[0])
        # check if splitting condition is true 
        if n>=self.nMinNode:
            retCondVal, retFeatureIndx = self.splitNode(X, y, m)
            leftNodeValIndx = X[:, retFeatureIndx] < retCondVal
            # build tree recrusively
            leftTree = self.buildDecisionTreeRecursive(X[leftNodeValIndx],y[leftNodeValIndx], depth+1)
            rightTree = self.buildDecisionTreeRecursive(X[~leftNodeValIndx],y[~leftNodeValIndx] , depth+1)
            return TreeNode(retFeatureIndx, retCondVal, leftTree, rightTree)
        
        # else condition 
        z = list(y)
        # put the value that has the most count
        return TreeNode(value=max(z, key=z.count))
        
    # recursive tree traversal
    def traverseTree(self, x, currNode):
        # if leaf node 
        if currNode.value!=None: 
            return currNode.value
        if x[currNode.featureIndex]<=currNode.conditionVal:
            return self.traverseTree(x, currNode.leftNode)
        else:
            return self.traverseTree(x, currNode.rightNode)

    def predict(self, X):
        return [self.traverseTree(x, self.root) for x in X]
    


In [33]:
def loadAndRunIris():
    # load the dataset
    df = pd.read_csv('iris.csv', header=None)
    # raw x and y
    rX = df.iloc[:, :-1].values
    ry = df.iloc[:, -1].values.reshape(-1,1)
    print(rX.shape)
    print(ry.shape)
    print(np.unique(ry))

    # split into training and testing sets
    X, xT, y, yT = train_test_split(rX, ry, test_size=0.2, random_state=50)
    
    # create and instance of decision tree
    classifier = MyDecisionTreeClassifier(3,X,y)

    # # make predictions on the testing data
    yP = classifier.predict(xT) 
    print("\nAccuracy: ",accuracy_score(yT, yP))

    return

loadAndRunIris()

(150, 4)
(150, 1)
['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']

Accuracy:  0.9666666666666667


In [34]:
from sklearn.model_selection import KFold

def doTenFoldValidation():
      # load the dataset
    df = pd.read_csv('iris.csv', header=None)

    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values.reshape(-1,1)

    nmin_range = [5, 10, 15, 20]

    # create an empty dataframe to store the results
    results_df = pd.DataFrame(columns=['nmin', 'Accuracy', 'Std Dev'])

    for nmin in nmin_range:
        # calculate the minimum number of instances
        nMinval = int(len(X) * (nmin / 100))

        totalAccuracy = []
        # divide data into k folds
        kFoldSet = KFold(n_splits=10, random_state=41,shuffle=True)
        for i, (trainIndx, testIndx) in enumerate(kFoldSet.split(X)):
            # divide data 
            xTrainSet, xTestSet = X[trainIndx],X[testIndx]
            yTrainSet, yTestSet = y[trainIndx],y[testIndx]
            classifier = MyDecisionTreeClassifier(nMinval,xTrainSet,yTrainSet)
            # predect 
            yP = classifier.predict(xTestSet) 
            totalAccuracy.append(accuracy_score(yTestSet,yP))

        accuracy = np.mean(totalAccuracy)
        std_dev = np.std(totalAccuracy)

        # add the results to the dataframe
        results_df = pd.concat([results_df, pd.DataFrame.from_records([{'nmin': nmin, 'Accuracy': accuracy, 'Std Dev': std_dev}])])

    # print the results dataframe
    print()
    print(results_df)

    return

doTenFoldValidation()


  nmin  Accuracy  Std Dev
0    5  0.946667     0.04
0   10  0.946667     0.04
0   15  0.946667     0.04
0   20  0.946667     0.04


In [35]:
def loadAndRunSpamBase():
    # load the dataset
    df = pd.read_csv('spambase.csv', header=None)
    # raw x and y
    rX = df.iloc[:, :-1].values
    ry = df.iloc[:, -1].values.reshape(-1,1)
    print(rX.shape)
    print(ry.shape)
    print(np.unique(ry))

    # split into training and testing sets
    X, xT, y, yT = train_test_split(rX, ry, test_size=0.2, random_state=50)
    
    # create and instance of decision tree
    classifier = MyDecisionTreeClassifier(3,X,y)

    # # make predictions on the testing data
    yP = classifier.predict(xT) 
    print("\nAccuracy: ",accuracy_score(yT, yP))

    return

loadAndRunSpamBase()

(4601, 57)
(4601, 1)
[0 1]

Accuracy:  0.9272529858849077


In [142]:
from sklearn.model_selection import KFold

def doTenFoldValidation2():
      # load the dataset
    df = pd.read_csv('spambase.csv', header=None)

    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values.reshape(-1,1)

    nmin_range = [5, 10, 15, 20, 25]

    # create an empty dataframe to store the results
    results_df = pd.DataFrame(columns=['nmin', 'Accuracy', 'Std Dev'])

    for nmin in nmin_range:
        # calculate the minimum number of instances
        nMinval = int(len(X) * (nmin / 100))

        totalAccuracy = []
        # divide data into k folds
        kFoldSet = KFold(n_splits=10, random_state=41,shuffle=True)
        for i, (trainIndx, testIndx) in enumerate(kFoldSet.split(X)):
            # divide data 
            xTrainSet, xTestSet = X[trainIndx],X[testIndx]
            yTrainSet, yTestSet = y[trainIndx],y[testIndx]
            classifier = MyDecisionTreeClassifier(nMinval,xTrainSet,yTrainSet)
            # predect 
            yP = classifier.predict(xTestSet) 
            totalAccuracy.append(accuracy_score(yTestSet,yP))

        accuracy = np.mean(totalAccuracy)
        std_dev = np.std(totalAccuracy)
        print('nmin', nmin, 'Accuracy',accuracy, 'Std Dev',std_dev)

        # add the results to the dataframe
        results_df = pd.concat([results_df, pd.DataFrame.from_records([{'nmin': nmin, 'Accuracy': accuracy, 'Std Dev': std_dev}])])

    # print the results dataframe
    print()
    print(results_df)

    return

doTenFoldValidation2()

nmin 5 Accuracy 0.9019800999717063 Std Dev 0.019246776673454327
nmin 10 Accuracy 0.8945892671885316 Std Dev 0.022147873345992265
nmin 15 Accuracy 0.8780675280580967 Std Dev 0.02373490398152029
nmin 20 Accuracy 0.8428614543053852 Std Dev 0.03411897686536198
nmin 25 Accuracy 0.8239484108271243 Std Dev 0.01615110738524305

  nmin  Accuracy   Std Dev
0    5  0.901980  0.019247
0   10  0.894589  0.022148
0   15  0.878068  0.023735
0   20  0.842861  0.034119
0   25  0.823948  0.016151
